In [36]:
# import numpy as np
# ## Define a class Convolution Layer, which is initialized with the various required params:
# class Convolution_Layer():
    
#     def __init__(self,input_image , filter_size, bias=True, stride=1, padding=0, dilation=1):
#         # For an untrained layer, set random initial filter weights
#         self.input_image=input_image
#         self.filter_size=filter_size
#         self.bias=True
#         self.stride=stride
#         self.padding=padding
#         self.dilation=dilation
#         self.input_channel=3
        
#         #setting random filter weights
#         self.filter_weights=np.random.rand(self.filter_size,self.filter_size)
#         # print(self.filter_weights)
#     def forward(self,input_image):
#         # Input Proprocess(According to pad etc.) Input will be of size (Batch_size, in_channels, inp_height, inp_width)
#         batch_size=input_image.shape[0]
#         input_size=input_image.shape[1]
#         #reshaping the image into 10000*3*32*32
#         input_image=input_image.reshape((batch_size,self.input_channel,32,32))
        
#         out_height = (32 + 2 * self.padding - self.dilation * (self.filter_size - 1) - 1) // self.stride + 1
#         out_width = (32 + 2 * self.padding - self.dilation * (self.filter_size - 1) - 1) // self.stride + 1
        
#         #creating the output image matrix
#         output = np.zeros((batch_size, self.input_channel, out_height, out_width))
#         # print(out_height)
#         # Reminder: Save Input for backward-prop
#         # Simple Conv operation:
#         # Loop over every location in inp_height * inp_width for the whole batch
#         print(input_image)
#         for batch in range(batch_size):
#             for channel in range(self.input_channel):
#                 for height in  range(out_height):
#                     for width in range(out_width):
#                         current_image_part=input_image[batch,channel,height*self.stride:height*self.stride+self.filter_size,width*self.stride:width*self.stride+self.filter_size]
#                         output[batch, channel, height, width] = np.sum(current_image_part * self.filter_weights[channel]) + (self.bias[channel] if self.bias is not None else 0)
        
    
#         # Output will be of the size (Batch_size, out_channels, out_height, out_width)
#         return output
    
#     def backward(self, grad_of_output_size):
        
#         # Naive Implementation
#         # Speed is not a concern
#         # Hint: gradients from each independant operation can be summed
        
#         #  return gradient of the size of the weight kernel
#         return grad
    
#     def set_weights(self, new_weights):
#         ## Replace the set of weights with the given 'new_weights'
#         ## use this for setting weights for blurring, bilateral filtering etc. 
#         pass

In [2]:
import numpy as np
from numba import njit
class Convolution_Layer():
    
    def __init__(self, filter_size, bias=True, stride=1, padding=0, dilation=1):
        self.filter_size = filter_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.bias = bias
        self.input_channel=3
        # Initialize filter weights randomly
        self.weights = np.random.randn(filter_size, filter_size)
        # self.batch_size=input_image.shape[0]
        # If bias is True, initialize bias terms
        if bias:
            self.bias = np.random.randn(1)
        else:
            self.bias = None
    # @njit
    def forward(self, input_image):
        input_image=input_image.reshape((10000,self.input_channel,32,32))
        batch_size, in_channels, inp_height, inp_width = input_image.shape
        out_channels = self.weights.shape[0]
        
        # Calculate output dimensions
        out_height = int(((inp_height + 2*self.padding - self.dilation*(self.filter_size - 1) - 1) / self.stride) + 1)
        out_width = int(((inp_width + 2*self.padding - self.dilation*(self.filter_size - 1) - 1) / self.stride) + 1)
        
        # Padding input image
        padded_input = np.pad(input_image, ((0,0), (0,0), (self.padding,self.padding), (self.padding,self.padding)), mode='constant')
        
        # Initialize output
        output = np.zeros((batch_size, out_channels, out_height, out_width))
        
        # Convolution
        for b in range(batch_size):
            for c_out in range(out_channels):
                for h_out in range(out_height):
                    for w_out in range(out_width):
                        h_start = h_out * self.stride
                        h_end = h_start + self.filter_size * self.dilation
                        w_start = w_out * self.stride
                        w_end = w_start + self.filter_size * self.dilation
                        
                        receptive_field = padded_input[b, :, h_start:h_end:self.dilation, w_start:w_end:self.dilation]
                        output[b, c_out, h_out, w_out] = np.sum(receptive_field * self.weights[c_out]) + self.bias
        
        return output
    
    def backward(self, grad_of_output_size):
        batch_size, out_channels, out_height, out_width = grad_of_output_size.shape
        _, in_channels, inp_height, inp_width = self.input_shape
        
        # Initialize gradients
        grad_weights = np.zeros_like(self.weights)
        grad_bias = np.zeros_like(self.bias)
        grad_input = np.zeros(self.input_shape)
        
        # Compute gradients
        for b in range(batch_size):
            for c_out in range(out_channels):
                for h_out in range(out_height):
                    for w_out in range(out_width):
                        h_start = h_out * self.stride
                        h_end = h_start + self.filter_size * self.dilation
                        w_start = w_out * self.stride
                        w_end = w_start + self.filter_size * self.dilation
                        
                        receptive_field = self.input_image[b, :, h_start:h_end:self.dilation, w_start:w_end:self.dilation]
                        
                        grad_weights[c_out] += grad_of_output_size[b, c_out, h_out, w_out] * receptive_field
                        grad_bias += grad_of_output_size[b, c_out, h_out, w_out]
                        grad_input[b, :, h_start:h_end:self.dilation, w_start:w_end:self.dilation] += grad_of_output_size[b, c_out, h_out, w_out] * self.weights[c_out]
        
        return grad_input, grad_weights, grad_bias
    
    def set_weights(self, new_weights):
        self.weights = new_weights


In [3]:
sol=Convolution_Layer(filter_size=3)

In [4]:
import pickle
def load_CIFAR():
    file_path='cifar-10-batches-py\data_batch_1'
    with open(file_path,'rb') as file:
        data=pickle.load(file,encoding='bytes')
    # for key in data.keys():
        # print(key)
    image_data=data[b'data']
    return image_data
input_data=load_CIFAR()

In [5]:
input_data.shape

(10000, 3072)

In [6]:
import matplotlib.pyplot as plt
res=sol.forward(input_data[:5])
print(res)

ValueError: cannot reshape array of size 15360 into shape (10000,3,32,32)

In [ ]:
res[0][0]

In [76]:
# import numpy as np

# class Convolution_Layer():
    
#     def __init__(self, input_data, filter_size, bias=True, stride=1, padding=0, dilation=1):
#         self.input_channels = input.shape[1]
#         self.filter_size = filter_size
#         self.bias = bias
#         self.stride = stride
#         self.padding = padding
#         self.dilation = dilation
        
#         # Initialize filter weights with random values
#         self.weights = np.random.randn(self.input_channels, filter_size, filter_size)
        
#         # Initialize bias if needed
#         if bias:
#             self.bias = np.zeros((self.input_channels,))
#         else:
#             self.bias = None
    
#     def forward(self, input):
#         batch_size, in_channels, in_height, in_width = input.shape
#         out_height = (in_height + 2 * self.padding - self.dilation * (self.filter_size - 1) - 1) // self.stride + 1
#         out_width = (in_width + 2 * self.padding - self.dilation * (self.filter_size - 1) - 1) // self.stride + 1
        
#         # Apply padding to the input
#         padded_input = np.pad(input, ((0, 0), (0, 0), (self.padding, self.padding), (self.padding, self.padding)), mode='constant')
        
#         output = np.zeros((batch_size, self.input_channels, out_height, out_width))
        
#         for b in range(batch_size):
#             for ic in range(self.input_channels):
#                 for oh in range(out_height):
#                     for ow in range(out_width):
#                         # Extract the current receptive field
#                         receptive_field = padded_input[b, ic, oh*self.stride:oh*self.stride+self.filter_size, ow*self.stride:ow*self.stride+self.filter_size]
#                         # Perform convolution
#                         output[b, ic, oh, ow] = np.sum(receptive_field * self.weights[ic]) + (self.bias[ic] if self.bias is not None else 0)
        
#         return output
    
#     def backward(self, grad_of_output_size):
#         batch_size, input_channels, out_height, out_width = grad_of_output_size.shape
        
#         grad_of_input = np.zeros_like(grad_of_output_size)
#         grad_of_weights = np.zeros_like(self.weights)
#         grad_of_bias = np.zeros_like(self.bias) if self.bias is not None else None
        
#         for b in range(batch_size):
#             for ic in range(self.input_channels):
#                 for oh in range(out_height):
#                     for ow in range(out_width):
#                         # Extract the current receptive field from the input
#                         receptive_field = input[b, ic, oh*self.stride:oh*self.stride+self.filter_size, ow*self.stride:ow*self.stride+self.filter_size]
#                         # Compute gradient of input
#                         grad_of_input[b, ic, oh*self.stride:oh*self.stride+self.filter_size, ow*self.stride:ow*self.stride+self.filter_size] += self.weights[ic] * grad_of_output_size[b, ic, oh, ow]
#                         # Compute gradient of weights
#                         grad_of_weights[ic] += receptive_field * grad_of_output_size[b, ic, oh, ow]
#                         # Compute gradient of bias
#                         if self.bias is not None:
#                             grad_of_bias[ic] += grad_of_output_size[b, ic, oh, ow]
        
#         return grad_of_input, grad_of_weights, grad_of_bias
    
#     def set_weights(self, new_weights):
#         self.weights = new_weights


In [48]:
load_CIFAR()

(10000, 3072)
